In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [54]:
out='../output/'
inp='../input/monthly/'
pref='fpw'

In [55]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,Min,,
2,Max,,
3,Average,,
4,Total,,
...,...,...,...
744,(2021/12/31) 23-24,53.50,20
745,Min,44.97,0
746,Max,185,50
747,Average,89.39024193548387096774193548,13.489247311827956989247311828


In [56]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
5,(2017/03/06) 04-05,29.00,50.0
6,(2017/03/06) 05-06,29.00,50.0
7,(2017/03/06) 06-07,65.00,0.0
8,(2017/03/06) 07-08,97.21,0.0
9,(2017/03/06) 08-09,97.21,0.0
...,...,...,...
740,(2021/12/31) 19-20,185.00,0.0
741,(2021/12/31) 20-21,110.00,0.0
742,(2021/12/31) 21-22,110.00,0.0
743,(2021/12/31) 22-23,53.50,20.0


In [57]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
5,(2017/03/06) 04-05,29.00,50.0,2017-03-06,4
6,(2017/03/06) 05-06,29.00,50.0,2017-03-06,5
7,(2017/03/06) 06-07,65.00,0.0,2017-03-06,6
8,(2017/03/06) 07-08,97.21,0.0,2017-03-06,7
9,(2017/03/06) 08-09,97.21,0.0,2017-03-06,8
...,...,...,...,...,...
740,(2021/12/31) 19-20,185.00,0.0,2021-12-31,19
741,(2021/12/31) 20-21,110.00,0.0,2021-12-31,20
742,(2021/12/31) 21-22,110.00,0.0,2021-12-31,21
743,(2021/12/31) 22-23,53.50,20.0,2021-12-31,22


In [58]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
5,29.00,50.0,2017-03-06,4
6,29.00,50.0,2017-03-06,5
7,65.00,0.0,2017-03-06,6
8,97.21,0.0,2017-03-06,7
9,97.21,0.0,2017-03-06,8
...,...,...,...,...
740,185.00,0.0,2021-12-31,19
741,110.00,0.0,2021-12-31,20
742,110.00,0.0,2021-12-31,21
743,53.50,20.0,2021-12-31,22


In [59]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0
2017-03-06,29.00,50.0,5,2017,3,6,0
2017-03-06,65.00,0.0,6,2017,3,6,0
2017-03-06,97.21,0.0,7,2017,3,6,0
2017-03-06,97.21,0.0,8,2017,3,6,0
...,...,...,...,...,...,...,...
2021-12-31,185.00,0.0,19,2021,12,31,4
2021-12-31,110.00,0.0,20,2021,12,31,4
2021-12-31,110.00,0.0,21,2021,12,31,4


In [60]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily['price_morning_max']=hourly.loc[hourly['hour']<11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_evening_max']=hourly.loc[hourly['hour']>=11][['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_standard']=hourly.loc[(hourly['hour']>=12) & (hourly['hour']<=15)][['year','month','day','price']].groupby(['year','month','day']).mean()
daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2021 12    27     80.0   4280.0             53.5     185.00       53.5   
           28    160.0   8560.0             53.5     185.00       53.5   
           29     80.0   4280.0             53.5     185.00       53.5   
           30     80.0   4280.0             53.5     185.00       53.5   
           31     80.0   4280.0             53.5     185.00       53.5   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 3     6     65.852500       50.0        0.0   10.000000   
           7     59.710417       50.0        0.0   16.666667   
           8     59.710417       50.0        0.0   16.666667   
           9     59.710417       50.0        0.0   16.666667   
           10    65.852500       50.0        0.0   10.000000   
...                    ...        ...        ...         ...   
2021 12    27   117.450000       20.0        0.0    4.000000   
           28   106.791667       20.0        0.0    6.666667   
           29   117.450000       20.0        0.0    4.000000   
           30   117.450000       20.0        0.0    4.000000   
           31   117.450000       20.0        0.0    4.000000   

                price_morning_max  price_evening_max  price_standard  \
year month day                                                         
2017 3     6                97.21              97.21            65.0   
           7                97.21              97.21            65.0   
           8                97.21              97.21            65.0   
           9                97.21              97.21            65.0   
           10               97.21              97.21            65.0   
...                           ...                ...             ...   
2021 12    27              185.00             185.00           110.0   
           28              185.00             185.00           110.0   
           29              185.00             185.00           110.0   
           30              185.00             185.00           110.0   
           31              185.00             185.00           110.0   

                     date  
year month day             
2017 3     6   2017-03-06  
           7   2017-03-07  
           8   2017-03-08  
           9   2017-03-09  
           10  2017-03-10  
...                   ...  
2021 12    27  2021-12-27  
           28  2021-12-28  
           29  2021-12-29  
           30  2021-12-30  
           31  2021-12-31  

[1708 rows x 13 columns]

In [61]:
hourly['cat']='off'

if (pref=='dam'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2017-03-06,29.00,50.0,4,2017,3,6,0,1450.0,off
2017-03-06,29.00,50.0,5,2017,3,6,0,1450.0,standard
2017-03-06,65.00,0.0,6,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,7,2017,3,6,0,0.0,morning
2017-03-06,97.21,0.0,8,2017,3,6,0,0.0,morning
...,...,...,...,...,...,...,...,...,...
2021-12-31,185.00,0.0,19,2021,12,31,4,0.0,evening
2021-12-31,110.00,0.0,20,2021,12,31,4,0.0,off
2021-12-31,110.00,0.0,21,2021,12,31,4,0.0,off


In [62]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume    value  price_weight_av  price_max  price_min  \
year month day                                                           
2017 3     6     200.0   5800.0             29.0      97.21       29.0   
           7     400.0  11600.0             29.0      97.21       29.0   
           8     400.0  11600.0             29.0      97.21       29.0   
           9     400.0  11600.0             29.0      97.21       29.0   
           10    200.0   5800.0             29.0      97.21       29.0   
...                ...      ...              ...        ...        ...   
2021 12    27     80.0   4280.0             53.5     185.00       53.5   
           28    160.0   8560.0             53.5     185.00       53.5   
           29     80.0   4280.0             53.5     185.00       53.5   
           30     80.0   4280.0             53.5     185.00       53.5   
           31     80.0   4280.0             53.5     185.00       53.5   

                price_mean  power_max  power_min  power_mean  \
year month day                                                 
2017 3     6     65.852500       50.0        0.0   10.000000   
           7     59.710417       50.0        0.0   16.666667   
           8     59.710417       50.0        0.0   16.666667   
           9     59.710417       50.0        0.0   16.666667   
           10    65.852500       50.0        0.0   10.000000   
...                    ...        ...        ...         ...   
2021 12    27   117.450000       20.0        0.0    4.000000   
           28   106.791667       20.0        0.0    6.666667   
           29   117.450000       20.0        0.0    4.000000   
           30   117.450000       20.0        0.0    4.000000   
           31   117.450000       20.0        0.0    4.000000   

                price_morning_max  ...  morning_hour_count  \
year month day                     ...                       
2017 3     6                97.21  ...                 3.0   
           7                97.21  ...                 3.0   
           8                97.21  ...                 3.0   
           9                97.21  ...                 3.0   
           10               97.21  ...                 3.0   
...                           ...  ...                 ...   
2021 12    27              185.00  ...                 3.0   
           28              185.00  ...                 3.0   
           29              185.00  ...                 3.0   
           30              185.00  ...                 3.0   
           31              185.00  ...                 3.0   

                evening_hour_count off_volume  standard_volume  \
year month day                                                   
2017 3     6                   2.0      150.0             50.0   
           7                   2.0      350.0             50.0   
           8                   2.0      350.0             50.0   
           9                   2.0      350.0             50.0   
           10                  2.0      150.0             50.0   
...                            ...        ...              ...   
2021 12    27                  2.0       60.0             20.0   
           28                  2.0      140.0             20.0   
           29                  2.0       60.0             20.0   
           30                  2.0       60.0             20.0   
           31                  2.0       60.0             20.0   

                morning_volume  evening_volume  off_value  standard_value  \
year month day                                                              
2017 3     6               0.0             0.0     4350.0          1450.0   
           7               0.0             0.0    10150.0          1450.0   
           8               0.0             0.0    10150.0          1450.0   
           9               0.0             0.0    10150.0          1450.0   
           10              0.0             0.0     4350.0          1450.0   
...            

In [63]:
daily.to_csv(out + pref + '_daily.csv')

In [64]:
hourly.to_csv(out + pref + '_hourly.csv')

In [65]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2017-03-06,4,29.00,50.0,off
2017-03-06,5,29.00,50.0,standard
2017-03-06,6,65.00,0.0,morning
2017-03-06,7,97.21,0.0,morning
2017-03-06,8,97.21,0.0,morning
...,...,...,...,...
2021-12-31,19,185.00,0.0,evening
2021-12-31,20,110.00,0.0,off
2021-12-31,21,110.00,0.0,off


In [66]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [67]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,480.730769,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.588812,33.946194,61.625498,86.514416,115.321589,24.752613,11.700000,19.868778,7.550000
4,2017,7,462.677419,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,20.210081,...,42.0,44.364721,32.046585,59.895533,79.061769,110.973696,24.497238,10.317460,18.755102,5.476190
5,2017,8,457.580645,22350.982258,64.280500,28.61,51.959191,23.419355,14.354839,19.978763,...,46.0,48.845996,32.627280,57.921955,82.467739,99.018571,21.474926,16.405797,19.771318,14.608696
6,2017,9,437.583333,19796.830000,63.146000,26.93,50.736536,21.000000,12.583333,18.965694,...,42.0,45.241280,31.487692,57.501670,75.761157,94.614187,21.286550,12.619048,19.091837,8.928571
7,2017,10,637.109677,29922.428645,60.922500,26.92,48.886422,29.032258,22.554839,27.565376,...,44.0,46.965899,31.352767,58.422616,76.447836,88.679545,29.242938,22.969697,28.235484,20.000000
8,2017,11,660.423333,28101.312300,59.438500,26.14,48.175603,38.596667,22.306667,28.336250,...,44.0,42.550453,28.730520,55.714666,76.915099,85.168890,35.746108,22.227273,21.691532,23.340909
9,2017,12,624.487097,28725.856194,64.220000,26.61,49.478970,32.809677,20.435484,27.004651,...,42.0,45.999119,31.008901,59.088798,78.054909,89.241905,29.997790,21.920635,25.628163,20.000000


In [68]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2017,3,597.631579,18470.265789,65.852500,29.00,49.938662,50.000000,9.473684,25.076754,...,30.0,30.905773,27.494558,34.148967,64.340852,86.158317,41.674641,7.822222,12.750000,6.733333
1,2017,4,1148.500000,38363.365000,44.465833,23.40,34.944417,64.900000,37.000000,50.404167,...,4.0,33.403017,24.330980,41.851838,76.297844,87.970000,59.745455,44.533333,30.874074,51.800000
2,2017,5,959.395238,42475.300905,57.075833,23.54,42.601155,61.290476,30.352381,40.792698,...,30.0,44.272995,26.819791,47.257466,79.193597,88.723370,41.441463,50.655556,34.235088,60.633333
3,2017,6,480.730769,23358.136923,71.232500,29.95,58.039212,28.461538,10.423077,20.528205,...,40.0,48.588812,33.946194,61.625498,86.514416,115.321589,24.752613,11.700000,19.868778,7.550000
4,2017,7,462.677419,20526.554677,70.355000,25.05,52.592774,22.838710,11.580645,20.210081,...,42.0,44.364721,32.046585,59.895533,79.061769,110.973696,24.497238,10.317460,18.755102,5.476190
5,2017,8,457.580645,22350.982258,64.280500,28.61,51.959191,23.419355,14.354839,19.978763,...,46.0,48.845996,32.627280,57.921955,82.467739,99.018571,21.474926,16.405797,19.771318,14.608696
6,2017,9,437.583333,19796.830000,63.146000,26.93,50.736536,21.000000,12.583333,18.965694,...,42.0,45.241280,31.487692,57.501670,75.761157,94.614187,21.286550,12.619048,19.091837,8.928571
7,2017,10,637.109677,29922.428645,60.922500,26.92,48.886422,29.032258,22.554839,27.565376,...,44.0,46.965899,31.352767,58.422616,76.447836,88.679545,29.242938,22.969697,28.235484,20.000000
8,2017,11,660.423333,28101.312300,59.438500,26.14,48.175603,38.596667,22.306667,28.336250,...,44.0,42.550453,28.730520,55.714666,76.915099,85.168890,35.746108,22.227273,21.691532,23.340909
9,2017,12,624.487097,28725.856194,64.220000,26.61,49.478970,32.809677,20.435484,27.004651,...,42.0,45.999119,31.008901,59.088798,78.054909,89.241905,29.997790,21.920635,25.628163,20.000000


In [69]:
monthly=monthly.set_index(['year','month'])

In [70]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [71]:
monthly.to_csv(out + pref + '_monthly.csv')

In [72]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [73]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,price_morning_max,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,12858.639234,52.905673,106.339355,38.316935,64.103270,15.245161,3.884677,10.288918,106.339355,...,95.50,398617.81625,344.75,254.75,67.50,45.0,11.776515,10.976113,5.392685,2.086646
1,2,10494.731264,63.009692,116.042170,40.484245,75.019848,14.111321,2.093396,7.679693,116.042170,...,0.00,278110.37850,307.75,215.50,56.25,37.5,10.776669,5.704035,1.720139,0.000000
2,3,10693.139441,43.717938,109.787832,36.123916,64.836252,20.606993,4.020979,11.221031,109.787832,...,40.40,305823.78800,322.20,234.00,61.80,41.2,20.946855,5.659563,2.424076,1.346667
3,4,14387.392177,39.695270,99.564194,31.688468,57.863964,23.585484,5.634677,14.432453,99.564194,...,217.88,356807.32600,281.20,201.80,53.40,35.6,25.184925,15.507457,14.741948,14.376494
4,5,16299.497483,37.200936,96.938414,29.579103,55.234248,29.296552,8.718621,17.405138,96.938414,...,590.96,472685.42700,329.60,235.40,61.80,41.2,23.511218,13.820844,15.504348,17.285963
5,6,19482.922521,43.268905,132.180068,34.696849,70.659254,36.750685,6.981507,20.150017,132.180068,...,63.00,568901.33760,330.80,239.40,63.00,42.0,31.278247,11.148157,4.169149,1.571905
6,7,18215.547019,46.589385,140.012387,37.616774,77.842331,30.968387,5.889032,16.517038,140.012387,...,171.40,564681.95760,348.20,252.80,66.60,44.4,27.480286,7.688665,4.572188,3.945238
7,8,19721.194103,52.666810,137.784065,37.434065,74.211419,28.167742,6.705806,16.885323,137.784065,...,333.20,611357.01720,348.20,252.80,66.60,44.4,23.610524,11.511842,8.316864,7.243478
8,9,23889.007993,48.928979,108.100267,33.143133,62.677024,39.917333,8.128000,21.847050,108.100267,...,383.00,716670.23980,343.20,242.80,63.60,42.4,31.814781,13.026594,10.048384,9.481667
9,10,16217.772703,47.755877,93.753806,33.464452,60.883656,19.440645,7.306452,14.521247,93.753806,...,214.00,502750.95380,348.20,252.80,66.60,44.4,17.918042,13.428743,7.825692,4.826087


In [74]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [75]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume        value
month hour                                    
1     0      38.381087  11.568478   509.468380
      1      38.381087  11.568478   509.468380
      2      38.381087  11.568478   509.468380
      3      38.381087  11.568478   509.468380
      4      38.316935  11.626613   513.657411
...                ...        ...          ...
12    19    126.787806  12.840000   725.605355
      20     71.233355  21.669032  1190.360852
      21     71.233355  21.669032  1190.360852
      22     38.137871  42.378710  1755.612090
      23     38.137871  42.378710  1755.612090

[288 rows x 3 columns]

In [76]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [77]:
from statsmodels.tsa.stattools import pacf

In [78]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,0.710433,0.715717,0.724551,0.820001
2,-0.062035,-0.180439,-0.167632,-0.269846
3,0.150392,0.304554,0.288528,0.105231
4,0.239200,0.185731,0.217226,0.199250
5,0.014624,0.164882,0.120018,0.002106
6,0.072994,0.098485,0.140089,0.014281
7,-0.051680,-0.180208,-0.207821,-0.098676
8,-0.095742,0.030375,0.010690,0.036693
9,-0.224485,-0.355744,-0.362925,-0.254035


In [79]:
pacfs.to_csv(out + pref + '_pacfs.csv')